# Setup

In [75]:
from langchain_openai import OpenAI
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.globals import set_llm_cache
import os
import json
import hashlib
import yaml

with open('config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)

os.environ['OPENAI_API_KEY'] = config['OPENAI_API_KEY']

# Memória

In [76]:
openai = OpenAI(model_name='gpt-3.5-turbo-instruct')
set_llm_cache(InMemoryCache())

In [77]:
prompt = 'Me diga em poucas palavras o sentido da vida'
response1 = openai.invoke(prompt)
print('Primeira resposta (API Chamada):', response1)

Primeira resposta (API Chamada): 

O sentido da vida é algo subjetivo e pode variar de pessoa para pessoa. Para alguns, pode ser encontrar felicidade e realização pessoal, para outros pode ser ajudar os outros e deixar um legado positivo. O importante é descobrir o que faz você se sentir completo e feliz, e buscar isso da melhor forma possível.


In [78]:
response2 = openai.invoke(prompt)
print('Segunda chamada (Cache):', response2)

Segunda chamada (Cache): 

O sentido da vida é algo subjetivo e pode variar de pessoa para pessoa. Para alguns, pode ser encontrar felicidade e realização pessoal, para outros pode ser ajudar os outros e deixar um legado positivo. O importante é descobrir o que faz você se sentir completo e feliz, e buscar isso da melhor forma possível.


# Banco de dados (SQLite)

In [79]:
set_llm_cache(SQLiteCache(database_path='cache/openai_cache.db'))
prompt = 'Me diga em poucas palavras quem foi Kratos'

In [80]:
response1 = openai.invoke(prompt)
print('Primeira resposta (API Chamada):', response1)

Primeira resposta (API Chamada): 

Kratos é um personagem fictício dos jogos eletrônicos da série God of War, criada pela empresa Santa Monica Studio. Ele é um guerreiro espartano extremamente poderoso que busca vingança contra os deuses do Olimpo após ser traído por eles e perder sua família. Com habilidades sobre-humanas e uma personalidade impiedosa, Kratos é conhecido por sua brutalidade e determinação em alcançar seus objetivos.


In [81]:
response2 = openai.invoke(prompt)
print('Segunda resposta (Cache):', response2)

Primeira resposta (Cache): 

Kratos é um personagem fictício dos jogos eletrônicos da série God of War, criada pela empresa Santa Monica Studio. Ele é um guerreiro espartano extremamente poderoso que busca vingança contra os deuses do Olimpo após ser traído por eles e perder sua família. Com habilidades sobre-humanas e uma personalidade impiedosa, Kratos é conhecido por sua brutalidade e determinação em alcançar seus objetivos.


# Personalizado

Classe personalizada com os métodos esperados pelo langchain para realizar o cachin, podendo ser usada outra abordagem além do JSON, que foi escolhida apenas para exemplificar.

In [91]:
class SimpleDiskCache:
    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)
    
    def _get_cache_path(self, key):
        hashed_key = hashlib.md5(key.encode()).hexdigest()
        return os.path.join(self.cache_dir, f'{hashed_key}.json')
    
    def lookup(self, key, llm_string):
        cache_path = self._get_cache_path(key)
        if os.path.exists(cache_path):
            with open(cache_path, 'r') as f:
                return json.load(f)
        return None
    
    def update(self, key, value, llm_string):
        cache_path = self._get_cache_path(key)
        with open(cache_path, 'w') as f:
            json.dump(value, f)

In [92]:
cache = SimpleDiskCache('cache')
set_llm_cache(cache)
prompt = 'Me diga em poucas palavras quem foi Sora de Kingdom Hearts'

In [97]:
def invoke_with_cache(llm, prompt, cache):
    cached_response = cache.lookup(prompt, '')
    if cached_response:
        print('Usando cache personalizado:\n')
        return cached_response
    response = llm.invoke(prompt)
    cache.update(prompt, response, '')
    return response

In [100]:
resp1 = invoke_with_cache(openai, prompt, cache)

print('Primeira resposta (API Chamada):', resp1)

Primeira resposta (API Chamada): 

Sora é o protagonista da série de jogos Kingdom Hearts. Ele é um jovem corajoso e amigável, que viaja pelos mundos da Disney e da Square Enix para proteger a paz e a luz do coração de todos. Ele possui uma Keyblade, uma arma mágica que utiliza para lutar contra as trevas e os Heartless. Sora é conhecido por sua determinação e lealdade aos seus amigos, e é o escolhido para ser o herói que salvará todos os mundos das forças do mal.


In [102]:
resp2 = invoke_with_cache(openai, prompt, cache)

print('Segunda resposta (Cache):', resp2)

Usando cache personalizado:

Segunda resposta (Cache): 

Sora é o protagonista da série de jogos Kingdom Hearts. Ele é um jovem corajoso e amigável, que viaja pelos mundos da Disney e da Square Enix para proteger a paz e a luz do coração de todos. Ele possui uma Keyblade, uma arma mágica que utiliza para lutar contra as trevas e os Heartless. Sora é conhecido por sua determinação e lealdade aos seus amigos, e é o escolhido para ser o herói que salvará todos os mundos das forças do mal.
